In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt


plt.rcParams["figure.figsize"] = (6, 3)
plt.rcParams['axes.grid'] = False
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False
plt.rcParams["font.size"] = 12
plt.rcParams["savefig.bbox"] = "tight"
plt.rcParams['figure.facecolor'] = 'white'

# Read in and simplify shapes
- probably three conditions in dataset:
1) boxes is empty
2) Boxes is a bounding box (four points)
3) Boxes is a segmentation mask of many points

In [ ]:
import json

json_fp = "./data/annotations/rotifer-food-capture.v2i.coco/train/_annotations.coco.json"
with open(json_fp, 'r') as f:
    json_data = json.load(f)

image_ids = [image['id'] for image in json_data['images']]
file_names = [image['file_name'] for image in json_data['images']]
file_names = [fn.split("_jpg")[0] for fn in file_names]    

bboxes, img_id = [], []
for annotation in json_data["annotations"]:
    bboxes.append(annotation["bbox"])
    img_id.append(annotation["image_id"])

print("images", len(image_ids), "fns", len(file_names), "\nannot: (bb)", len(bboxes), "ids", len(img_id))

#d = {"ids" : image_ids, "file_names" : file_names,}

#pd.DataFrame.from_dict(d)
#df = df.sort_values(by="file_names")

In [ ]:
df_imgs = pd.DataFrame({"ids" : image_ids, "fns" : file_names})
df_annots = pd.DataFrame({"ids" : img_id, "bbox" : bboxes})
df_merged = pd.merge(df_imgs, df_annots, on="ids")
df_filtered = df_merged.dropna(subset="bbox")
df = df_filtered.sort_values(by="fns").reset_index(drop=True)
df

In [ ]:
food_capture_dir = "./data/images/2024_0323_075154_003_food_capture_frames/"

df["img_fp"] = [os.path.join(food_capture_dir, f + ".jpg") for f in df.fns.tolist()]
from PIL import Image
from utils.piv_utils import plot_bbox

bbox, fp = df[["bbox", "img_fp"]].T.values
df["cm"] = [(np.mean([bb[0], bb[0] + bb[2]]), np.mean([bb[1], bb[1] + bb[3]])) for bb in bbox]

bbox[0], fp[0]

In [ ]:
for nth in range(0, 100, 10):
    plt.figure()
    plt.imshow(Image.open(fp[nth]))
    plt.plot([bbox[nth][0], bbox[nth][0] + bbox[nth][2]],
             [bbox[nth][1], bbox[nth][1] + bbox[nth][3]], "+r")
    plt.plot(np.mean([bbox[nth][0], bbox[nth][0] + bbox[nth][2]]),
             np.mean([bbox[nth][1], bbox[nth][1] + bbox[nth][3]]), "go")
    plt.plot(df.cm.iloc[nth][0], df.cm.iloc[nth][1], "k.")
    #plot_bbox(bbox[0])

In [ ]:
df.to_csv("./data/csvs/metadata_food_capture.csv", index=False)

# 2. 2024_0409_183844_105

In [42]:
import json

json_fp = "./data/annotations/rotifer-food-capture-0409-105.v1i.coco/train/_annotations.coco.json"
with open(json_fp, 'r') as f:
    json_data = json.load(f)

image_ids = [image['id'] for image in json_data['images']]
file_names = [image['file_name'] for image in json_data['images']]
file_names = [fn.split("_jpg")[0] for fn in file_names]    

bboxes, img_id = [], []
for annotation in json_data["annotations"]:
    bboxes.append(annotation["bbox"])
    img_id.append(annotation["image_id"])

print("images", len(image_ids), "fns", len(file_names), "\nannot: (bb)", len(bboxes), "ids", len(img_id))
# are all BBOXES taken???

images 278 fns 278 
annot: (bb) 408 ids 408


In [43]:
df_imgs = pd.DataFrame({"ids" : image_ids, "fns" : file_names})
df_annots = pd.DataFrame({"ids" : img_id, "bbox" : bboxes})
df_merged = pd.merge(df_imgs, df_annots, on="ids")
df_filtered = df_merged.dropna(subset="bbox")
df = df_filtered.sort_values(by="fns").reset_index(drop=True)
df

,ids,fns,bbox
0,14,frame_00060,"[96, 169, 29.42, 19.95]"
1,132,frame_00061,"[113, 168, 26.19, 21.31]"
2,84,frame_00062,"[126, 171, 26.19, 21.31]"
3,125,frame_00063,"[141, 172, 26.19, 21.31]"
4,106,frame_00064,"[159, 175, 26.19, 21.31]"
...,...,...,...
403,38,frame_01909,"[426, 229, 19.94, 17.84]"
404,20,frame_01910,"[428, 230, 19.94, 17.84]"
405,115,frame_01911,"[429, 229, 19.94, 17.84]"
406,255,frame_01912,"[430, 227, 19.94, 17.84]"


In [44]:
df["img_idx"] = [int(s[-1]) for s in df.fns.str.split("_")]
df["frame"] = df["img_idx"]
df = df.set_index("img_idx")
df["particle"] = None

capture_frames = [[60,95],[1305,1349],[1553,1611],[1737,1777],[1817,1899],[1863,1913]]

for nth, cut in enumerate(capture_frames):
        idx = np.arange(cut[0], cut[1]+1)
        df.loc[df.index.isin(idx), "particle"] = int(nth) + 1


for nth, cut in enumerate(capture_frames):
        idx = np.arange(cut[0], cut[1]+1)
        fcounts = sum(df.frame.between(cut[0], cut[1]+1))
        #df.loc[df.index.isin(idx), "particle"] = int(nth) + 1
        print(len(idx), fcounts)

df.to_csv("./data/csvs/metadata_food_capture_0409_105_prefinal.csv")
# manually adjusted the particle number in metadata_food_capture_0409_105_prefinal_manualadjust.csv
# unfortunatly 1817-1899 and the last one overlap, so probably lost this one somehow?!

In [49]:
df_adjusted = pd.read_csv("./data/csvs/metadata_food_capture_0409_105_prefinal_manualadjust.csv")
df_adjusted["bbox"] = df_adjusted["bbox"].apply(eval)

In [50]:
food_capture_dir = "./data/images/2024_0409_183844_105_food_capture_frames/"

df_adjusted["img_fp"] = [os.path.join(food_capture_dir, f + ".jpg") for f in df_adjusted.fns.tolist()]
from PIL import Image
from utils.piv_utils import plot_bbox

bbox, fp = df_adjusted[["bbox", "img_fp"]].T.values

df_adjusted["cm"] = [(round(np.mean([bb[0], bb[0] + bb[2]]),2),
             round(np.mean([bb[1], bb[1] + bb[3]]),2)) for bb in bbox]

bbox[0], fp[0]

([96, 169, 29.42, 19.95],
 './data/images/2024_0409_183844_105_food_capture_frames/frame_00060.jpg')

In [53]:
df_adjusted.to_csv("./data/csvs/metadata_food_capture_0409_105_final.csv")


In [ ]:
for nth in range(0, 400, 20):
    plt.figure()
    plt.imshow(Image.open(fp[nth]))
    plt.plot([bbox[nth][0], bbox[nth][0] + bbox[nth][2]],
             [bbox[nth][1], bbox[nth][1] + bbox[nth][3]], "+r")
    plt.plot(np.mean([bbox[nth][0], bbox[nth][0] + bbox[nth][2]]),
             np.mean([bbox[nth][1], bbox[nth][1] + bbox[nth][3]]), "go")
    plt.plot(df_adjusted.cm.iloc[nth][0], df_adjusted.cm.iloc[nth][1], "k.")
    #plot_bbox(bbox[0])

# 3. 2024_0331_007_trimmed

In [10]:
import json

json_fp = "./data/annotations/2024_0331_007_food_capture.v1i.coco/train/_annotations.coco.json"
with open(json_fp, 'r') as f:
    json_data = json.load(f)

image_ids = [image['id'] for image in json_data['images']]
file_names = [image['file_name'] for image in json_data['images']]
file_names = [fn.split("_jpg")[0] for fn in file_names]    

bboxes, img_id, cat_id = [], [], []
for annotation in json_data["annotations"]:
    bboxes.append(annotation["bbox"])
    img_id.append(annotation["image_id"])
    cat_id.append(annotation["category_id"])

print("images", len(image_ids), "fns", len(file_names), "\nannot: (bb)", len(bboxes), "ids", len(img_id), "category ids", np.unique(cat_id))

images 328 fns 328 
annot: (bb) 329 ids 329 category ids [ 1  2  3  4  5  6  7  8  9 10 11]


In [11]:
df_imgs = pd.DataFrame({"ids" : image_ids, "fns" : file_names})
df_annots = pd.DataFrame({"ids" : img_id, "bbox" : bboxes, "particle" : cat_id})
df_merged = pd.merge(df_imgs, df_annots, on="ids")
df_filtered = df_merged.dropna(subset="bbox")
df = df_filtered.sort_values(by="fns").reset_index(drop=True)
df

,ids,fns,bbox,particle
0,282,frame_00000,"[509, 469, 21.8, 29.06]",1
1,84,frame_00001,"[510, 460, 20.71, 26.74]",1
2,221,frame_00002,"[510, 448, 20.71, 26.74]",1
3,119,frame_00003,"[510, 436, 20.71, 26.74]",1
4,173,frame_00004,"[512, 420, 20.71, 26.74]",1
...,...,...,...,...
324,57,frame_00680,"[511, 69, 16.82, 19.31]",3
325,140,frame_00681,"[509, 71, 16.82, 19.31]",3
326,10,frame_00682,"[509, 69, 16.82, 19.31]",3
327,6,frame_00683,"[511, 67, 16.82, 19.31]",3


In [12]:
df.particle.unique()

array([ 1,  4,  5,  6,  7,  9,  8, 10, 11,  2,  3])

In [13]:
food_capture_dir = "./data/images/2024_0331_091926_007_trimmed_food_capture_frames/"

df["img_fp"] = [os.path.join(food_capture_dir, f + ".jpg") for f in df.fns.tolist()]
from PIL import Image
from utils.piv_utils import plot_bbox

bbox, fp = df[["bbox", "img_fp"]].T.values
df["cm"] = [(np.mean([bb[0], bb[0] + bb[2]]), np.mean([bb[1], bb[1] + bb[3]])) for bb in bbox]

bbox[0], fp[0]

([509, 469, 21.8, 29.06],
 './data/images/2024_0331_091926_007_trimmed_food_capture_frames/frame_00000.jpg')

In [ ]:
for nth in range(0, 100, 10):
    plt.figure()
    plt.imshow(Image.open(fp[nth]))
    plt.plot([bbox[nth][0], bbox[nth][0] + bbox[nth][2]],
             [bbox[nth][1], bbox[nth][1] + bbox[nth][3]], "+r")
    plt.plot(np.mean([bbox[nth][0], bbox[nth][0] + bbox[nth][2]]),
             np.mean([bbox[nth][1], bbox[nth][1] + bbox[nth][3]]), "go")
    plt.plot(df.cm.iloc[nth][0], df.cm.iloc[nth][1], "k.")
    #plot_bbox(bbox[0])

In [15]:
df.to_csv("./data/csvs/metadata_food_capture_2024_0331_091926_007_trimmed.csv", index=False)

# 4. 2024_0403_115801_028.MP4 - 120Hz

In [2]:
import json

json_fp = "./data/annotations/2024_0403_115801_028-120hz-capt.v1i.coco/train/_annotations.coco.json"
with open(json_fp, 'r') as f:
    json_data = json.load(f)

image_ids = [image['id'] for image in json_data['images']]
file_names = [image['file_name'] for image in json_data['images']]
file_names = [fn.split("_jpg")[0] for fn in file_names]    

bboxes, img_id, cat_id = [], [], []
for annotation in json_data["annotations"]:
    bboxes.append(annotation["bbox"])
    img_id.append(annotation["image_id"])
    cat_id.append(annotation["category_id"])

print("images", len(image_ids), "fns", len(file_names), "\nannot: (bb)", len(bboxes), "ids", len(img_id), "category ids", np.unique(cat_id))

images 202 fns 202 
annot: (bb) 202 ids 202 category ids [1 2]


In [3]:
df_imgs = pd.DataFrame({"ids" : image_ids, "fns" : file_names})
df_annots = pd.DataFrame({"ids" : img_id, "bbox" : bboxes, "particle" : cat_id})
df_merged = pd.merge(df_imgs, df_annots, on="ids")
df_filtered = df_merged.dropna(subset="bbox")
df = df_filtered.sort_values(by="fns").reset_index(drop=True)
df

,ids,fns,bbox,particle
0,108,frame_00140,"[468, 319, 12.96, 13.84]",1
1,71,frame_00141,"[463, 320, 12.96, 13.84]",1
2,4,frame_00142,"[457, 323, 12.96, 13.84]",1
3,30,frame_00143,"[450, 326, 12.96, 13.84]",1
4,119,frame_00144,"[445, 329, 12.96, 13.84]",1
...,...,...,...,...
197,117,frame_01126,"[320, 396, 9.31, 9.96]",2
198,9,frame_01127,"[320, 396, 9.31, 9.96]",2
199,113,frame_01128,"[320, 397, 9.31, 9.96]",2
200,183,frame_01129,"[321, 399, 9.31, 9.96]",2


In [4]:
food_capture_dir = "./data/images/2024_0403_115801_028/"

df["img_fp"] = [os.path.join(food_capture_dir, f + ".jpg") for f in df.fns.tolist()]
from PIL import Image
from utils.piv_utils import plot_bbox

bbox, fp = df[["bbox", "img_fp"]].T.values
df["cm"] = [(np.mean([bb[0], bb[0] + bb[2]]), np.mean([bb[1], bb[1] + bb[3]])) for bb in bbox]

bbox[0], fp[0]

([468, 319, 12.96, 13.84],
 './data/images/2024_0403_115801_028/frame_00140.jpg')

In [ ]:
for nth in range(0, 100, 10):
    plt.figure()
    plt.imshow(Image.open(fp[nth]))
    plt.plot([bbox[nth][0], bbox[nth][0] + bbox[nth][2]],
             [bbox[nth][1], bbox[nth][1] + bbox[nth][3]], "+r")
    plt.plot(np.mean([bbox[nth][0], bbox[nth][0] + bbox[nth][2]]),
             np.mean([bbox[nth][1], bbox[nth][1] + bbox[nth][3]]), "go")
    plt.plot(df.cm.iloc[nth][0], df.cm.iloc[nth][1], "k.")
    #plot_bbox(bbox[0])

In [6]:
df.to_csv("./data/csvs/metadata_food_capture_2024_0403_115801_028.csv", index=False)